In [1]:
import tensorflow as tf
import os 
import numpy as np
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random 
import cv2
import tensorflow.keras.backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import Sequence
import threading

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
class dataloader:
    def __init__(self):
        self.mainDIR = os.listdir('.')
        self.files = os.listdir('./Dataset/')
        self.masterDataset = pd.read_csv('dataSet.csv')
        print("MAINDATASET : ", self.masterDataset.shape)
        for file in self.mainDIR:
            try:
                int(file)
                self.files.append(file)
            except Exception:
                pass
        
        self.mainCSV = pd.DataFrame(columns = ['X', 'Y', #Of the actual data
                                                'frameName', # ADD THE FILE NAME HERE TO STOP DUPLICATION
                                                'Xcam', 'Ycam', 'Orientation',
                                                'faceXmin', 'faceYmin',
                                                'faceXmax', 'faceYmax', 'faceValid',
                                                
                                                'leftEyeXmin', 'leftEyeYmin',
                                                'leftEyeXmax', 'leftEyeYmax', 'leftEyeValid',
                                                
                                                'rightEyeXmin', 'rightEyeYmin',
                                                'rightEyeXmax', 'rightEyeYmax', 'rightEyeValid',
                                                
                                                # 25 X 25 grid
                                                'faceGirdXmin', 'faceGridYmin',
                                                'faceGridXmax', 'faceGridYmax', 'faceGridValid',
                                                
                                                'screenHeight', 'screenWidth', 'Device'
                                                ])
        
        self.masterDataset = pd.read_csv('dataSet.csv')
        self.splitcolumn = int(self.masterDataset.shape[0] * 0.9)
        self.trainCSV = self.masterDataset.head(self.splitcolumn)
        self.testCSV = self.masterDataset.tail(self.masterDataset.shape[0] - self.splitcolumn)
    
    def mainCSVcreator(self):
        print("IN MAIN CSV CREATOR")
        self.files = os.listdir('./Dataset/')
        first = True
        n = 0
        for file in self.files:
            print(file)
            n += 1
            try:
                face = pd.read_json('./Dataset/' + file + '/appleFace.json')
                leftEye = pd.read_json('./Dataset/' + file + '/appleLeftEye.json')
                rightEye = pd.read_json('./Dataset/' + file + '/appleRightEye.json')
                dotInfo = pd.read_json('./Dataset/' + file + '/dotInfo.json')
                faceGrid = pd.read_json('./Dataset/' + file + '/faceGrid.json')
                frames = pd.read_json('./Dataset/' + file + '/frames.json')
                info = pd.read_json('./Dataset/' + file + '/info.json', typ='series')
                motion = pd.read_json('./Dataset/' + file + '/motion.json')
                screen = pd.read_json('./Dataset/' + file + '/screen.json')
                
                numImages = os.listdir('./Dataset/' + file + '/frames/')
                if frames[0].shape[0] != len(numImages):
                    print("NOT SAME NUMBER")
                    break
            except Exception as ex:
                print("ECEPTION  :  " + str(ex))
                break
            
            tempDF = pd.DataFrame()
            
            tempDF['X'] = dotInfo['XPts']
            tempDF['Y'] = dotInfo['YPts']
            tempDF['frameName'] = file + '/frames/' + frames[0]
            tempDF['Xcam'] = dotInfo['XCam']
            tempDF['Ycam'] = dotInfo['YCam']
            tempDF['Orientation'] = screen['Orientation']
            
            tempDF['faceXmin'] = face['X']
            tempDF['faceYmin'] = face['Y']
            tempDF['faceXmax'] = face['X'] + face['W']
            tempDF['faceYmax'] = face['Y'] + face['H']
            tempDF['faceValid'] = face['IsValid']
            
            tempDF['leftEyeXmin'] = leftEye['X']
            tempDF['leftEyeYmin'] = leftEye['Y']
            tempDF['leftEyeXmax'] = leftEye['X'] + leftEye['W']
            tempDF['leftEyeYmax'] = leftEye['Y'] + leftEye['H']
            tempDF['leftEyeValid'] = leftEye['IsValid']
            
            tempDF['rightEyeXmin'] = rightEye['X']
            tempDF['rightEyeYmin'] = rightEye['Y']
            tempDF['rightEyeXmax'] = rightEye['X'] + rightEye['W']
            tempDF['rightEyeYmax'] = rightEye['Y'] + rightEye['H']
            tempDF['rightEyeValid'] = rightEye['IsValid']
            
            tempDF['faceGridXmin'] = faceGrid['X']
            tempDF['faceGridYmin'] = faceGrid['Y']
            tempDF['faceGridXmax'] = faceGrid['X'] + faceGrid['W']
            tempDF['faceGridYmax'] = faceGrid['Y'] + faceGrid['H']
            tempDF['faceGridValid'] = faceGrid['IsValid']
            
            tempDF['screenHeight'] = screen['H']
            tempDF['screenWidth'] = screen['W']
            tempDF['Device'] = info['DeviceName']
            
            print(tempDF.shape)
            
            for i in range(tempDF.shape[0]):
                #print(tempDF.iloc[i]['faceValid'], tempDF[i]['leftEyeValid'], tempDF[i]['rightEyeValid'], tempDF[i]['faceGridValid'])
                if (tempDF.loc[i]['faceValid'] == 0 or tempDF.loc[i]['leftEyeValid'] == 0 or
                    tempDF.loc[i]['rightEyeValid'] == 0 or tempDF.loc[i]['faceGridValid'] == 0 or 
                    tempDF.loc[i]['leftEyeXmax'] < 0 or tempDF.loc[i]['leftEyeYmax'] < 0 or 
                    tempDF.loc[i]['leftEyeXmin'] < 0 or tempDF.loc[i]['leftEyeYmin'] < 0 or 
                    tempDF.loc[i]['rightEyeXmax'] < 0 or tempDF.loc[i]['rightEyeYmax'] < 0 or 
                    tempDF.loc[i]['rightEyeXmin'] < 0 or tempDF.loc[i]['rightEyeYmin'] < 0 or 
                    tempDF.loc[i]['faceXmax'] < 0 or tempDF.loc[i]['faceYmax'] < 0 or 
                    tempDF.loc[i]['faceXmin'] < 0 or tempDF.loc[i]['faceYmin'] < 0):
                    tempDF = tempDF.drop(i, axis = 0)
                
            
            if first is True:
                self.mainCSV = tempDF
                tempDF = None
                first = False 
            else:
                print(self.mainCSV.shape)
                self.mainCSV = self.mainCSV.append(tempDF)
                tempDF = None
                
            if n % 50 == 0:
                self.mainCSV.to_csv('dataSet.csv')
        
        self.mainCSV.to_csv('dataSet.csv')
    
    def getEpochs_train(self, size):
        return self.trainCSV.shape[0] // size - 8000
    
    def getEpochs_test(self, size):
        return self.testCSV.shape[0] // size - 800

    def load_data_train(self, batchSize):
        masterDataset = self.trainCSV
        
        while True:
            leftEyeImage, rightEyeImage, faceImage, Grids = None, None, None, None
            rows = random.sample(range(0, masterDataset.shape[0]), batchSize)
            miniDataset = masterDataset.iloc[rows]
            
            trainY = miniDataset[['Xcam', 'Ycam']].to_numpy()
            
            faceImage = None
            First = True
            for i in range(batchSize):
                if First is True:
                    #print(miniDataset.iloc[i]['frameName'])
                    img = cv2.imread('./Dataset/' + miniDataset.iloc[i]['frameName'])
                    faceGrid = np.array([np.zeros((25,25))])
                    faceGrid[0][miniDataset.iloc[i]['faceGridYmin']:miniDataset.iloc[i]['faceGridYmax'], miniDataset.iloc[i]['faceGridXmin']:miniDataset.iloc[i]['faceGridXmax']] = 1
                    
                    Grids = faceGrid
                    faceImage = np.array([cv2.resize(img[int(miniDataset.iloc[i]['faceYmin']):int(miniDataset.iloc[i]['faceYmax']), int(miniDataset.iloc[i]['faceXmin']):int(miniDataset.iloc[i]['faceXmax']), :], (64,64), interpolation=cv2.INTER_AREA)  ])
                    leftEyeImage = np.array([cv2.resize(img[int(miniDataset.iloc[i]['leftEyeYmin']):int(miniDataset.iloc[i]['leftEyeYmax']), int(miniDataset.iloc[i]['leftEyeXmin']):int(miniDataset.iloc[i]['leftEyeXmax']), :], (64,64), interpolation=cv2.INTER_AREA)])
                    rightEyeImage = np.array([cv2.resize(img[int(miniDataset.iloc[i]['rightEyeYmin']):int(miniDataset.iloc[i]['rightEyeYmax']), int(miniDataset.iloc[i]['rightEyeXmin']):int(miniDataset.iloc[i]['rightEyeXmax']), :], (64,64), interpolation=cv2.INTER_AREA)])
                    First = False   
                else:
                    img = cv2.imread('./Dataset/' + miniDataset.iloc[i]['frameName'])
                    faceGrid = np.array([np.zeros((25,25))])
                    faceGrid[0][miniDataset.iloc[i]['faceGridYmin']:miniDataset.iloc[i]['faceGridYmax'], miniDataset.iloc[i]['faceGridXmin']:miniDataset.iloc[i]['faceGridXmax']] = 1
                    
                    #print(leftEyeImage.shape, rightEyeImage.shape, faceImage.shape, Grids.shape)
                    
                    Grids = np.append(Grids, faceGrid, axis = 0)
                    leftEyeImage = np.append(leftEyeImage, np.array([cv2.resize(img[int(miniDataset.iloc[i]['leftEyeYmin']):int(miniDataset.iloc[i]['leftEyeYmax']), int(miniDataset.iloc[i]['leftEyeXmin']):int(miniDataset.iloc[i]['leftEyeXmax']), :], (64,64), interpolation=cv2.INTER_AREA)]), axis = 0)
                    rightEyeImage = np.append(rightEyeImage, np.array([cv2.resize(img[int(miniDataset.iloc[i]['rightEyeYmin']):int(miniDataset.iloc[i]['rightEyeYmax']), int(miniDataset.iloc[i]['rightEyeXmin']):int(miniDataset.iloc[i]['rightEyeXmax']), :], (64,64), interpolation=cv2.INTER_AREA)]), axis = 0)
                    faceImage = np.append(faceImage, np.array([cv2.resize( img[int(miniDataset.iloc[i]['faceYmin']):int(miniDataset.iloc[i]['faceYmax']), int(miniDataset.iloc[i]['faceXmin']):int(miniDataset.iloc[i]['faceXmax']), :] , (64,64), interpolation=cv2.INTER_AREA)  ]), axis = 0)
                    
            #print("END : " , leftEyeImage.shape, rightEyeImage.shape, faceImage.shape, Grids.shape)
            yield [leftEyeImage, rightEyeImage, faceImage, Grids], trainY

    def load_data_test(self, batchSize):
        masterDataset = self.testCSV
        
        while True:
            leftEyeImage, rightEyeImage, faceImage, Grids = None, None, None, None
            rows = random.sample(range(0, masterDataset.shape[0]), batchSize)
            miniDataset = masterDataset.iloc[rows]
            
            trainY = miniDataset[['Xcam', 'Ycam']].to_numpy()
            
            faceImage = None
            First = True
            for i in range(batchSize):
                if First is True:
                    #print(miniDataset.iloc[i]['frameName'])
                    img = cv2.imread('./Dataset/' + miniDataset.iloc[i]['frameName'])
                    faceGrid = np.array([np.zeros((25,25))])
                    faceGrid[0][miniDataset.iloc[i]['faceGridYmin']:miniDataset.iloc[i]['faceGridYmax'], miniDataset.iloc[i]['faceGridXmin']:miniDataset.iloc[i]['faceGridXmax']] = 1
                    
                    Grids = faceGrid
                    faceImage = np.array([cv2.resize(img[int(miniDataset.iloc[i]['faceYmin']):int(miniDataset.iloc[i]['faceYmax']), int(miniDataset.iloc[i]['faceXmin']):int(miniDataset.iloc[i]['faceXmax']), :], (64,64), interpolation=cv2.INTER_AREA)  ])
                    leftEyeImage = np.array([cv2.resize(img[int(miniDataset.iloc[i]['leftEyeYmin']):int(miniDataset.iloc[i]['leftEyeYmax']), int(miniDataset.iloc[i]['leftEyeXmin']):int(miniDataset.iloc[i]['leftEyeXmax']), :], (64,64), interpolation=cv2.INTER_AREA)])
                    rightEyeImage = np.array([cv2.resize(img[int(miniDataset.iloc[i]['rightEyeYmin']):int(miniDataset.iloc[i]['rightEyeYmax']), int(miniDataset.iloc[i]['rightEyeXmin']):int(miniDataset.iloc[i]['rightEyeXmax']), :], (64,64), interpolation=cv2.INTER_AREA)])
                    First = False
                else:
                    img = cv2.imread('./Dataset/' + miniDataset.iloc[i]['frameName'])
                    faceGrid = np.array([np.zeros((25,25))])
                    faceGrid[0][miniDataset.iloc[i]['faceGridYmin']:miniDataset.iloc[i]['faceGridYmax'], miniDataset.iloc[i]['faceGridXmin']:miniDataset.iloc[i]['faceGridXmax']] = 1
                    
                    #print(leftEyeImage.shape, rightEyeImage.shape, faceImage.shape, Grids.shape)
                    
                    Grids = np.append(Grids, faceGrid, axis = 0)
                    leftEyeImage = np.append(leftEyeImage, np.array([cv2.resize(img[int(miniDataset.iloc[i]['leftEyeYmin']):int(miniDataset.iloc[i]['leftEyeYmax']), int(miniDataset.iloc[i]['leftEyeXmin']):int(miniDataset.iloc[i]['leftEyeXmax']), :], (64,64), interpolation=cv2.INTER_AREA)]), axis = 0)
                    rightEyeImage = np.append(rightEyeImage, np.array([cv2.resize(img[int(miniDataset.iloc[i]['rightEyeYmin']):int(miniDataset.iloc[i]['rightEyeYmax']), int(miniDataset.iloc[i]['rightEyeXmin']):int(miniDataset.iloc[i]['rightEyeXmax']), :], (64,64), interpolation=cv2.INTER_AREA)]), axis = 0)
                    faceImage = np.append(faceImage, np.array([cv2.resize( img[int(miniDataset.iloc[i]['faceYmin']):int(miniDataset.iloc[i]['faceYmax']), int(miniDataset.iloc[i]['faceXmin']):int(miniDataset.iloc[i]['faceXmax']), :] , (64,64), interpolation=cv2.INTER_AREA)  ]), axis = 0)
                    
            #print("END : " , leftEyeImage.shape, rightEyeImage.shape, faceImage.shape, Grids.shape)
            yield [leftEyeImage, rightEyeImage, faceImage, Grids], trainY

In [4]:
dl = dataloader()
#dl.mainCSVcreator()

('MAINDATASET : ', (1335663, 30))


In [5]:
class training:
    def __init__(self):
        self.activation = 'relu'
        self.optimizer = tf.keras.optimizers.Adam()
        
        #multiworker_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
        #multiworker_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy(tf.distribute.experimental.CollectiveCommunication.NCCL)
        #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
        
        self.preMadeModel = self.getModel()
        self.preMadeModel.compile(self.optimizer, loss = 'mse', metrics = [self.myAccuracy])
    
    def myAccuracy(self, y_true, y_pred):
        diff = K.abs(y_true-y_pred) #absolute difference between correct and predicted values
        correct = K.less(diff,0.05) #tensor with 0 for false values and 1 for true values
        return K.mean(correct) #sum all 1's and divide by the total.
    
    def getModel(self):
        leftEyeInput = tf.keras.layers.Input(shape = (64,64,3))
        a = tf.keras.layers.Conv2D(96, kernel_size = 11, strides = 4, activation = self.activation)(leftEyeInput)
        a = tf.keras.layers.MaxPooling2D(pool_size = 3)(a)
        a = tf.keras.layers.BatchNormalization()(a)
        a = tf.keras.layers.Flatten()(a)
        lModel = tf.keras.models.Model(inputs = leftEyeInput, outputs = a)

        rightEyeInput = tf.keras.layers.Input(shape = (64,64,3))
        b = tf.keras.layers.Conv2D(96, kernel_size = 11, strides = 4, activation = self.activation)(rightEyeInput)
        b = tf.keras.layers.MaxPooling2D(pool_size = 3)(b)
        b = tf.keras.layers.BatchNormalization()(b)
        b = tf.keras.layers.Flatten()(b)
        rModel = tf.keras.models.Model(inputs = rightEyeInput, outputs = b)


        outEyes = tf.keras.layers.concatenate([rModel.output, lModel.output])
        outEyes = tf.keras.layers.Dense(128, activation = self.activation)(outEyes)

        #Face
        faceInput = tf.keras.layers.Input(shape = (64,64,3))
        c = tf.keras.layers.Conv2D(96, kernel_size = 11, strides = 4, activation = self.activation)(faceInput)
        c = tf.keras.layers.MaxPooling2D(pool_size = 3)(c)
        c = tf.keras.layers.BatchNormalization()(c)
        c = tf.keras.layers.Flatten()(c)
        fModel = tf.keras.models.Model(inputs = faceInput, outputs = c)

        #Face Grid
        faceGridInput = tf.keras.layers.Input(shape = (25,25))
        d = tf.keras.layers.Flatten()(faceGridInput)
        gModel = tf.keras.models.Model(inputs = faceGridInput, outputs = d)

        outFace = tf.keras.layers.concatenate([fModel.output, gModel.output])
        outFace = tf.keras.layers.Dense(128, activation = self.activation)(outFace)

        finalModel = tf.keras.layers.concatenate([outFace, outEyes])
        finalModel = tf.keras.layers.Dense(128, activation = self.activation)(finalModel)
        finalModel = tf.keras.layers.Dropout(0.25)(finalModel)
        finalModel = tf.keras.layers.Dense(2)(finalModel)


        model = tf.keras.models.Model(inputs = [leftEyeInput, rightEyeInput, faceInput, faceGridInput], outputs = finalModel)

        return model

In [6]:
class trainGenSeq(Sequence):
    def __init__(self, batchSize):
        self.batchSize = batchSize
        
        self.masterDataset = pd.read_csv('dataSet.csv')
        self.splitcolumn = int(self.masterDataset.shape[0] * 0.9)
        self.trainCSV = self.masterDataset.head(self.splitcolumn)
    
    def __len__(self):
        return int(self.trainCSV.shape[0] // self.batchSize)
    
    def getLen(self):
        return int(self.trainCSV.shape[0] // self.batchSize)
    
    def __getitem__(self, trash):
        masterDataset = self.trainCSV
        leftEyeImage, rightEyeImage, faceImage, Grids = None, None, None, None
        rows = random.sample(range(0, masterDataset.shape[0]), batchSize)
        miniDataset = masterDataset.iloc[rows]
        
        trainY = miniDataset[['Xcam', 'Ycam']].to_numpy()
        
        faceImage = None
        First = True
        for i in range(batchSize):
            if First is True:
                #print(miniDataset.iloc[i]['frameName'])
                img = cv2.imread('./Dataset/' + miniDataset.iloc[i]['frameName'])
                faceGrid = np.array([np.zeros((25,25))])
                faceGrid[0][miniDataset.iloc[i]['faceGridYmin']:miniDataset.iloc[i]['faceGridYmax'], miniDataset.iloc[i]['faceGridXmin']:miniDataset.iloc[i]['faceGridXmax']] = 1
                
                Grids = faceGrid
                faceImage = np.array([cv2.resize(img[int(miniDataset.iloc[i]['faceYmin']):int(miniDataset.iloc[i]['faceYmax']), int(miniDataset.iloc[i]['faceXmin']):int(miniDataset.iloc[i]['faceXmax']), :], (64,64), interpolation=cv2.INTER_AREA)  ])
                leftEyeImage = np.array([cv2.resize(img[int(miniDataset.iloc[i]['leftEyeYmin']):int(miniDataset.iloc[i]['leftEyeYmax']), int(miniDataset.iloc[i]['leftEyeXmin']):int(miniDataset.iloc[i]['leftEyeXmax']), :], (64,64), interpolation=cv2.INTER_AREA)])
                rightEyeImage = np.array([cv2.resize(img[int(miniDataset.iloc[i]['rightEyeYmin']):int(miniDataset.iloc[i]['rightEyeYmax']), int(miniDataset.iloc[i]['rightEyeXmin']):int(miniDataset.iloc[i]['rightEyeXmax']), :], (64,64), interpolation=cv2.INTER_AREA)])
                First = False   
            else:
                img = cv2.imread('./Dataset/' + miniDataset.iloc[i]['frameName'])
                faceGrid = np.array([np.zeros((25,25))])
                faceGrid[0][miniDataset.iloc[i]['faceGridYmin']:miniDataset.iloc[i]['faceGridYmax'], miniDataset.iloc[i]['faceGridXmin']:miniDataset.iloc[i]['faceGridXmax']] = 1
                
                #print(leftEyeImage.shape, rightEyeImage.shape, faceImage.shape, Grids.shape)
                
                Grids = np.append(Grids, faceGrid, axis = 0)
                leftEyeImage = np.append(leftEyeImage, np.array([cv2.resize(img[int(miniDataset.iloc[i]['leftEyeYmin']):int(miniDataset.iloc[i]['leftEyeYmax']), int(miniDataset.iloc[i]['leftEyeXmin']):int(miniDataset.iloc[i]['leftEyeXmax']), :], (64,64), interpolation=cv2.INTER_AREA)]), axis = 0)
                rightEyeImage = np.append(rightEyeImage, np.array([cv2.resize(img[int(miniDataset.iloc[i]['rightEyeYmin']):int(miniDataset.iloc[i]['rightEyeYmax']), int(miniDataset.iloc[i]['rightEyeXmin']):int(miniDataset.iloc[i]['rightEyeXmax']), :], (64,64), interpolation=cv2.INTER_AREA)]), axis = 0)
                faceImage = np.append(faceImage, np.array([cv2.resize( img[int(miniDataset.iloc[i]['faceYmin']):int(miniDataset.iloc[i]['faceYmax']), int(miniDataset.iloc[i]['faceXmin']):int(miniDataset.iloc[i]['faceXmax']), :] , (64,64), interpolation=cv2.INTER_AREA)  ]), axis = 0)
                
        return [leftEyeImage, rightEyeImage, faceImage, Grids], trainY
    
class testGenSeq(Sequence):
    def __init__(self, batchSize):
        self.batchSize = batchSize
        
        self.masterDataset = pd.read_csv('dataSet.csv')
        self.splitcolumn = int(self.masterDataset.shape[0] * 0.9)
        self.testCSV = self.masterDataset.tail(self.masterDataset.shape[0] - self.splitcolumn)
    
    def __len__(self):
        return int(self.testCSV.shape[0] // self.batchSize)
    
    def getLen(self):
        return int(self.testCSV.shape[0] // self.batchSize)
    
    def __getitem__(self, trash):
        masterDataset = self.testCSV
        leftEyeImage, rightEyeImage, faceImage, Grids = None, None, None, None
        rows = random.sample(range(0, masterDataset.shape[0]), batchSize)
        miniDataset = masterDataset.iloc[rows]
        
        trainY = miniDataset[['Xcam', 'Ycam']].to_numpy()
        
        faceImage = None
        First = True
        for i in range(batchSize):
            if First is True:
                #print(miniDataset.iloc[i]['frameName'])
                img = cv2.imread('./Dataset/' + miniDataset.iloc[i]['frameName'])
                faceGrid = np.array([np.zeros((25,25))])
                faceGrid[0][miniDataset.iloc[i]['faceGridYmin']:miniDataset.iloc[i]['faceGridYmax'], miniDataset.iloc[i]['faceGridXmin']:miniDataset.iloc[i]['faceGridXmax']] = 1
                
                Grids = faceGrid
                faceImage = np.array([cv2.resize(img[int(miniDataset.iloc[i]['faceYmin']):int(miniDataset.iloc[i]['faceYmax']), int(miniDataset.iloc[i]['faceXmin']):int(miniDataset.iloc[i]['faceXmax']), :], (64,64), interpolation=cv2.INTER_AREA)  ])
                leftEyeImage = np.array([cv2.resize(img[int(miniDataset.iloc[i]['leftEyeYmin']):int(miniDataset.iloc[i]['leftEyeYmax']), int(miniDataset.iloc[i]['leftEyeXmin']):int(miniDataset.iloc[i]['leftEyeXmax']), :], (64,64), interpolation=cv2.INTER_AREA)])
                rightEyeImage = np.array([cv2.resize(img[int(miniDataset.iloc[i]['rightEyeYmin']):int(miniDataset.iloc[i]['rightEyeYmax']), int(miniDataset.iloc[i]['rightEyeXmin']):int(miniDataset.iloc[i]['rightEyeXmax']), :], (64,64), interpolation=cv2.INTER_AREA)])
                First = False   
            else:
                img = cv2.imread('./Dataset/' + miniDataset.iloc[i]['frameName'])
                faceGrid = np.array([np.zeros((25,25))])
                faceGrid[0][miniDataset.iloc[i]['faceGridYmin']:miniDataset.iloc[i]['faceGridYmax'], miniDataset.iloc[i]['faceGridXmin']:miniDataset.iloc[i]['faceGridXmax']] = 1
                
                #print(leftEyeImage.shape, rightEyeImage.shape, faceImage.shape, Grids.shape)
                
                Grids = np.append(Grids, faceGrid, axis = 0)
                leftEyeImage = np.append(leftEyeImage, np.array([cv2.resize(img[int(miniDataset.iloc[i]['leftEyeYmin']):int(miniDataset.iloc[i]['leftEyeYmax']), int(miniDataset.iloc[i]['leftEyeXmin']):int(miniDataset.iloc[i]['leftEyeXmax']), :], (64,64), interpolation=cv2.INTER_AREA)]), axis = 0)
                rightEyeImage = np.append(rightEyeImage, np.array([cv2.resize(img[int(miniDataset.iloc[i]['rightEyeYmin']):int(miniDataset.iloc[i]['rightEyeYmax']), int(miniDataset.iloc[i]['rightEyeXmin']):int(miniDataset.iloc[i]['rightEyeXmax']), :], (64,64), interpolation=cv2.INTER_AREA)]), axis = 0)
                faceImage = np.append(faceImage, np.array([cv2.resize( img[int(miniDataset.iloc[i]['faceYmin']):int(miniDataset.iloc[i]['faceYmax']), int(miniDataset.iloc[i]['faceXmin']):int(miniDataset.iloc[i]['faceXmax']), :] , (64,64), interpolation=cv2.INTER_AREA)  ]), axis = 0)
                
        return [leftEyeImage, rightEyeImage, faceImage, Grids], trainY
        

In [7]:
train = training()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
model = train.preMadeModel

In [ ]:
batchSize = 128

checkpoint_path = "training_64_Large/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

trainSeq = trainGenSeq(batchSize)
testSeq = testGenSeq(batchSize)

history = model.fit(           trainSeq,
                               #dl.load_data_train(batchSize),
                               epochs = 5,
                               steps_per_epoch = trainSeq.getLen(),
                               #steps_per_epoch = dl.getEpochs_train(batchSize),
                               verbose = 1,
                               #validation_data = dl.load_data_test(batchSize),
                               #validation_steps = dl.getEpochs_test(batchSize),
                               validation_data = testSeq,
                               validation_steps = testSeq.getLen(),
                               use_multiprocessing = True,
                               workers = 32
)

print("FINISED TRAINING")

model.save('largeModel_64.h5')
model.save_weights('largeWeights_64.h5')

Epoch 1/5
3827/9391 [===========>..................] - ETA: 25:39 - loss: 10.8464 - myAccuracy: 0.0144

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, color='red', label='Training loss')
plt.plot(epochs, val_loss, color='green', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()